<a href="https://colab.research.google.com/github/leahgriffith123/SN-Classifier/blob/master/Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Setup



**Google Authentication**
It should be possible to access all data if Leah_SURF folder is shared.

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

**General Purpose**

In [ ]:
import tqdm
import pandas as pd
import json
from bson.json_util import loads, dumps
import gzip
import os
import io
from typing import Union
import pathlib
import numpy as np
from astropy.io import fits
from astropy.time import Time
import datetime
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
!pip install xlrd
!pip install psycopg2
import psycopg2 as ps
from google.colab import files

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


**Kowalski Imports**

In [ ]:
!pip install git+https://github.com/dmitryduev/kowalski.git
!pip install pymongo

from IPython.core.display import display, HTML, JSON
import json
from bson.json_util import dumps, loads 


from penquins import Kowalski

  Cloning https://github.com/dmitryduev/kowalski.git to /tmp/pip-req-build-u04uor45
  Running command git clone -q https://github.com/dmitryduev/kowalski.git /tmp/pip-req-build-u04uor45
  Created wheel for penquins: filename=penquins-1.0.3-cp36-none-any.whl size=4706 sha256=610a8c963c49c42dc690755fb5d43c68f0d8ad47248a44c0f617f86b39914684
  Stored in directory: /tmp/pip-ephem-wheel-cache-dpluflff/wheels/bd/d1/4d/130550e0acb3c3dc15f45217f25ea42f669bbd43af6e32f2d1
Successfully built penquins


In [ ]:
secrets = {
    "kowalski": {
        "username": "leahgriffith",
        "password": "ExceptionallyStrongPa$$word"
    }
}

In [ ]:
k = Kowalski(username=secrets['kowalski']['username'], password=secrets['kowalski']['password'])


connection_ok = k.check_connection()
print(f'Connection OK: {connection_ok}')

Connection OK: True


# Make Triplets and Plotting

In [ ]:
def make_triplet(alert, normalize: bool = False):
    """
        Feed in alert packet
    """
    cutout_dict = dict()

    for cutout in ('science', 'template', 'difference'):
        cutout_data = loads(dumps([alert[f'cutout{cutout.capitalize()}']['stampData']]))[0]

        # unzip
        with gzip.open(io.BytesIO(cutout_data), 'rb') as f:
            with fits.open(io.BytesIO(f.read())) as hdu:
                data = hdu[0].data
                # replace nans with zeros
                cutout_dict[cutout] = np.nan_to_num(data)
                # normalize
                if normalize:
                    cutout_dict[cutout] /= np.linalg.norm(cutout_dict[cutout])

        # pad to 63x63 if smaller
        shape = cutout_dict[cutout].shape
        if shape != (63, 63):
            cutout_dict[cutout] = np.pad(cutout_dict[cutout], [(0, 63 - shape[0]), (0, 63 - shape[1])],
                                         mode='constant', constant_values=1e-9)

    triplet = np.zeros((63, 63, 3))
    triplet[:, :, 0] = cutout_dict['science']
    triplet[:, :, 1] = cutout_dict['template']
    triplet[:, :, 2] = cutout_dict['difference']

    return triplet

In [ ]:
def plot_triplet(tr):
    fig = plt.figure(figsize=(8, 2), dpi=120)
    ax1 = fig.add_subplot(131)
    ax1.axis('off')
    ax1.imshow(tr[:, :, 0], origin='upper', cmap=plt.cm.bone, norm=LogNorm())
    ax1.title.set_text('Science')
    ax2 = fig.add_subplot(132)
    ax2.axis('off')
    ax2.imshow(tr[:, :, 1], origin='upper', cmap=plt.cm.bone, norm=LogNorm())
    ax2.title.set_text('Reference')
    ax3 = fig.add_subplot(133)
    ax3.axis('off')
    ax3.imshow(tr[:, :, 2], origin='upper', cmap=plt.cm.bone)
    ax3.title.set_text('Difference')
    plt.show()

# Querying

In [ ]:
objectIdExcel = "https://github.com/leahgriffith123/SN-Classifier/blob/master/ObjectIDsExcelData.xlsx?raw=true"
ObjectIdData = pd.read_excel(objectIdExcel)
agnList = list(ObjectIdData['agnObjectIdColumn'])
sniList = list(ObjectIdData['sniObjectIdColumn'])
sniiList = list(ObjectIdData['sniiObjectIdColumn'])
bogusList = list(ObjectIdData['bogusObjectIdColumn'])

In [ ]:
#!git clone https://github.com/leahgriffith123/SN-Classifier.git
#!git init .
folder_id = '1Xa0y6DpQRxhElFpapXlrTCuQXBM8-fah'
def classification(typeObject):
  for indObjectId in typeObject:
    q = {"query_type": "find",
     "query": {
         "catalog":"ZTF_alerts",
         "filter": {"objectId": indObjectId},
         "projection": {"_id": 0},
     }
     }
    r = k.query(query=q)
    alerts = r['result_data']['query_result']
    for i in alerts:
      triplet = make_triplet(i)
      filename = "'AGN" + indObjectId +"'.npy"
      np.save(filename, triplet)
    #files.download(filename)
    folder = drive.CreateFile({'parents': [{'id': folder_id}]})
    folder.SetContentFile(filename)
    folder.Upload()

# !git add "'ZTF19acxnemy'.npy"
# !git commit
# !git push origin master